In [ ]:
# Build the prompt with a hardcoded ReAct framework
prompt = f"""
Model Metadata:
{model_metadata}

You are provided with detailed contents from the subsection of MDD titled "{sub_section_name}".
This is the only context you may use. DO NOT incorporate any other information or your own prior knowledge.

Follow the steps of the ReAct framework below to reason through and answer the question:

1. Thought:
   - Analyze the provided context carefully.
   - Summarize the key points and details that are relevant.
2. Action:
   - Identify the specific information or process required by the question using only the above context.
3. Observation:
   - Reflect on your analysis and check that the context supports your conclusion.
4. Final Answer:
   - Based on your previous steps, provide a detailed, coherent, and complete answer to the following question.
   
Context:
{context}
"""

# Combine with your base system prompt and process guidelines
messages = [
    {"role": "system", "content": system_prompt_individual_answers},
    {"role": "system", "content": prompt}
]
outputs = []

# Process each guideline by sending it as the user message,
# generating the answer from the model, appending the result, and printing the output.
%%time
for guideline in guidelines:
    # Here we set the current user query
    messages = [{"role": "user", "content": guideline}]
    # Generate model output using a function that applies the template
    output = generate_chat_template(model=model, tokenizer=tokenizer, messages=messages, temperature=0.2, max_new_tokens=8000)
    # Append the assistant's output so that context accumulates if needed
    messages += [{"role": "assistant", "content": output}]
    print(guideline)
    print("".join(["#"]*15))
    print(output)
    print("\n\n")
    outputs.append(output)


In [ ]:
outputs = []

%%time
for guideline in guidelines:
    # Use the current guideline as the query for retrieval.
    search_results = vector_store.search(guideline, k=3)
    
    # Concatenate retrieved document chunks to form the metadata context.
    retrieved_context = ""
    for doc, score in search_results:
        retrieved_context += f"\n---\n{doc.content}\n"
    
    # Build a prompt incorporating the chain-of-thought instructions and the retrieved metadata context.
    prompt = f"""
Model Metadata:
{model_metadata}

You are provided with chunked metadata context below extracted from our documents.
Do NOT incorporate any external knowledge or your internal knowledge beyond this context.

Retrieved Metadata Context:
{retrieved_context}

Follow these steps:
1. Thought: Analyze the metadata context and the question.
2. Action: Reason using the provided context.
3. Observation: Reflect on the key points from the retrieved metadata.
4. Final Answer: Produce a detailed, coherent, and complete answer to the question.

Question: {guideline}
"""
    
    # Set up the messages for generate_chat_template.
    messages = [
        {"role": "system", "content": system_prompt_individual_answers},
        {"role": "system", "content": prompt}
    ]
    # Append the guideline again as a user message.
    messages.append({"role": "user", "content": guideline})
    
    # Generate the answer using the chat template function.
    output = generate_chat_template(model=model, tokenizer=tokenizer, messages=messages, temperature=0.2, max_new_tokens=8000)
    
    # Accumulate the assistant's output into the messages for potential context chaining.
    messages.append({"role": "assistant", "content": output})
    
    # Print and store the result.
    print(guideline)
    print("".join(["#"*15]))
    print(output)
    print("\n\n")
    outputs.append(output)
